In [1]:
# !pip install -q langchain==0.1.13
# !pip install -q pymupdf==1.23.19
# !pip install -q huggingface-hub==0.20.3
# !pip install -q faiss-cpu==1.7.4
# !pip install -q sentence-transformers==2.2.2
# !pip install -q openai==1.14.3

In [2]:
# Imports
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import langchain
import textwrap
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.llms import OpenAI
import warnings
warnings.filterwarnings('ignore')

## Chave GPT OpenAI
## sk-proj-OeYXnUYJhWZbag78bLujT3BlbkFJ7ChuikFtVQr9wGIftFPU

In [3]:
# Função para carregar o pdf
def dsa_carrega_pdf(file_path):
    # Cria uma instâncvia da classe PyMuPDFLoader, passando o caminho do arquivo PDF como argumento.
    loader = PyMuPDFLoader(file_path=file_path)
    
    # Usa o método 'load' do objeto 'loader' para carregar o conteúdo do PDF.
    # Isso retorna um objeto ou uma estrutura de dados contendo as páginas do PDF com seu conteúdo.
    docs = loader.load()
    
    # Retorna o conteúdo carregado do PDF.
    return docs

In [4]:
# Função para dividir os documentos em vários pedaços (chunks)
def dsa_split_docs(documents, chunk_size=1000, chunk_overlap=20):
    # Cria uma instância da classe RecursiveCharacterTextSplitter.
    # Esta classe divide textos longos em pedaços menores (chunks).
    # "chunk_size" define o tamanho de cada pedaço, e "chunk_overlap" define a sobreposição entre pedaços consecutivos.
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
    
    # Utiliza o método "split_documents" do objeto "text_splitter" para dividir o documento fornecido.
    # "documents" é uma variável que contém o texto ou conjunto de textos a serem divididos.
    chunks = text_splitter.split_documents(documents=documents)
    
    # Retorna os pedaços de texto resultantes da divisão.
    return chunks

In [5]:
# Carrega o modelo de embeddings
def dsa_carrega_embedding_model(model_path, normalize_embedding=True):
    # Retorna uma instância da classe HuggingFaceEmbeddings
    # "model_name" é o identificador do modelo de embeddings a ser carregado
    # "model_kwargs" é o dicionário de argumentos adicionais para a configuração do modelo, neste caso, definindo o dispositivo para "cpu".
    # "encode_kwargs" é um dicionário de argumentos para o método de codificação, aqui especificando se os embeddings devem ser normalizados.
    return HuggingFaceEmbeddings(model_name=model_path,
                                 model_kwargs={"device": "cpu"},
                                 encode_kwargs={"normalize_embeddings": normalize_embedding})

In [6]:
# Função para criar embeddings
def dsa_cria_embeddings(chunks, embedding_model, storing_path = "modelo/vectorstore"):
    # Cria um "vectorestore" (um índice FAISS) a partir dos documentos fornecidos.
    # "chunks" é a lista de segmentos de texto e "embeddings_model" é o modelo de mebedding utilizado para converter texto em embeddings.
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    
    # Salva o "vectorstore" criado em um caminho local especificado por "storing_path".
    # Isso permite a persistência do índice FAISS para uso futuro.
    vectorstore.save_local(storing_path)
    
    # Retorna o "vectorstore" criado, que contém os embeddings e pode ser usado para operações de busca e comparação de similaridade.
    return vectorstore

In [7]:
# Criando a chain
def dsa_load_qa_chain(retriever, llm, prompt):
    # Retorna uma instância da classe RetrievalQA.
    # Esta função lida com a cadeia de processos envolvidos em um sistema de Question Answering (QA).
    # "llm" refere-se ao modelo de linguagem de grande escala (como um modelo GPT ou BERT).
    # "retriever" é um componente usado para recuperar informações relevantes (como um mecanismo ded busca ou um retriver de documentos).
    # "chain_type" define o tipo de cadeia ou estratégia usada no processo de QA. Aqui, está definido como "stuff", um placeholder para um tipo real.
    # "return_source_documents": um booleano que, quando True, indica que os documentos fonte (ou seja, os documentos de onde as respostas são extraídas) devem ser retornados juntamente com as respostas.
    # "chain_type_kwargs" é um dicionário de argumentos adicionais específicos para o tipo de cadeia escolhido. Aqui, está passando "prompt" como argumento.
    return RetrievalQA.from_chain_type(llm=llm,
                                       retriever = retriever,
                                       chain_type = "stuff",
                                       return_source_documents = True,
                                       chain_type_kwargs = {"prompt": prompt})

In [9]:
# Função para obter as respostas do LLM (Large Languaage Model)
def dsa_get_response(query, chain):
    # Invoca a "chain" (cadeia de processamento, um pipeline de Question Answering) com a "query" fornecida.
    # "chain" é uma função que recebe uma consulta e retorna uma resposta, utilizando o LLM.
    response = chain({"query": query})
    
    # Utiliza a biblioteca textwrap para formatar a resposta. "textwrap.fill" quebra o texto da resposta em linhas de largura especificada (100 caracteres neste caso),
    # tornando mais fácil a leitura em ambientes como o Jupyter Notebook.
    wrapped_text = textwrap.fill(response["result"], width=100)
    
    # Imprime o texto formatado
    print(wrapped_text)

In [10]:
# Definindo a API da OpenAI
llm_api = OpenAI(openai_api_key="sk-proj-OeYXnUYJhWZbag78bLujT3BlbkFJ7ChuikFtVQr9wGIftFPU")

In [11]:
# Carrega o modelo Embedding
embed = dsa_carrega_embedding_model(model_path="all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [12]:
# Carrega o arquivo pdf
docs = dsa_carrega_pdf(file_path="a_psicologia_das_emocoes_o_fascinio_do_rosto_humano_by_a_freitas.pdf")

In [13]:
# Divide o arquivo em chunks
documents = dsa_split_docs(documents=docs)

In [14]:
# Cria o vectorestore
vectorstore = dsa_cria_embeddings(documents, embed)

In [19]:
# Converte o vectorstore para um retriever
retriever = vectorstore.as_retriever()

In [21]:
# Cria o template
template="""
### System:
Você é um assistente pessoal. Você tem que responder as perguntas do usuário \
usando apenas o contexto fornecido a você. Se você não sabe a resposta, \
apenas diga que você não sabe. Não tente inventar uma resposta.

### Context:
{context}

### User:
{question}

### Response:
"""

In [16]:
# Cria o template
template="""
### System:
Você é um assistente pessoal. Você tem que responder as perguntas do usuário \
usando apenas o contexto fornecido a você. Se você não sabe a resposta, \
apenas diga que você não sabe. Não tente inventar uma resposta.

### Context:
Sou um vendedor e quero aprender sobre as emoções para poder vender melhor, \
aprendendo sobre as emoções posso vender um produto ideal para meu cliente, \
dependendo da emoção apresentada.

### User:
Bruno

### Response:
"""

In [22]:
# Criando o prompt a partir do template
prompt = PromptTemplate.from_template(template)

In [23]:
# Criando a chain (pipeline)
dsa_chain = dsa_load_qa_chain(retriever, llm_api, prompt)

In [25]:
# Interagindo com o assistente
dsa_get_response("Sobre o que se trata o livro?", dsa_chain)

# dsa_get_response("O que posso aprender com ele?", dsa_chain)

# dsa_get_response("Quantas páginas esse livro tem?", dsa_chain)

# Fim

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}